In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import yfinance as yf
import warnings
import mpl_finance as mpf
# from scipy import integrate
# from scipy.stats import t

from ipywidgets import widgets
from ipywidgets import interact, interact_manual
from IPython.display import display

In [130]:
%matplotlib inline
warnings.filterwarnings("ignore")

global df
global ticker

def controller(sender):
    ticker = sender.value
    df = pd.read_csv(ticker + '.csv')
    rsi.main(df, ticker)
#     macd.main(df, ticker)
#     parabolic_sar.main(df, ticker)
#     heikin_ashi.main(df, ticker)

@interact
def get_ticker():
    text = widgets.Text()
    display(text)
    text.on_submit(controller)

## RSI

In [128]:
class rsi:
    def smma(series,n):

        output=[series[0]]

        for i in range(1,len(series)):
            temp=output[-1]*(n-1)+series[i]
            output.append(temp/n)

        return output
    
    def rsi(data,n):

        delta=data.diff().dropna()
        up=np.where(delta>0,delta,0)
        down=np.where(delta<0,-delta,0)
        rs=np.divide(rsi.smma(up,n),rsi.smma(down,n))
        output=100-100/(1+rs)

        return output[n-1:]

    def signal_generation(df,n):
        
        df['rsi']=0.0
        df['rsi'][n:] = rsi.rsi(df['close'],n)
        df['positions'] = np.select([df['rsi']<30,df['rsi']>70], [1,-1],default=0)
        df['signals'] = df['positions'].diff()

        return df[n:]

    def plot(new,ticker):

        fig = plt.figure(figsize=(20,10))
        ax1 = plt.subplot2grid((2,1), (0,0))

        new['close'].plot(label=ticker)
        ax1.plot(new.loc[new['signals']==1].index,
                new['close'][new['signals']==1],
                label='LONG',lw=0,marker='^',c='g')
        ax1.plot(new.loc[new['signals']==-1].index,
                new['close'][new['signals']==-1],
                label='SHORT',lw=0,marker='v',c='r')


        plt.legend(loc='best')
        plt.grid(True)
        plt.title('Positions')
        plt.xlabel('time')
        plt.ylabel('price')

        ax2 = plt.subplot2grid((2,1), (1,0))

        new['rsi'].plot(label='relative strength index',c='#522e75')
        ax2.fill_between(new.index,30,70,alpha=0.5,color='#f22f08')

        ax2.text(new.index[-60],75,'overbought',color='#594346',size=12.5)
        ax2.text(new.index[-45],25,'oversold',color='#594346',size=12.5)

        plt.xlabel('time')
        plt.ylabel('value')
        plt.title('RSI')
        plt.legend(loc='best')
        plt.grid(True)
        plt.show()

    def main(df, ticker): 
        
        %matplotlib inline
        
        slicer=114670
        new=df[slicer:].copy()
        new = new.reset_index()

        new = rsi.signal_generation(new, n=14)
        rsi.plot(new,ticker)
    
def controller(sender):
    ticker = sender.value
    df = pd.read_csv(ticker + '.csv')
    rsi.main(df, ticker)
#     macd.main(df, ticker)
#     parabolic_sar.main(df, ticker)
#     heikin_ashi.main(df, ticker)

@interact
def get_ticker():
    
    
    mylink = widgets.jslink((a, 'value'), (b, 'value'))
    
    
    text.on_submit(controller)

## MACD

In [70]:
class macd:
    def macd(signals):
        
        signals['ma1']=signals['close'].rolling(window=ma1,min_periods=1,center=False).mean()
        signals['ma2']=signals['close'].rolling(window=ma2,min_periods=1,center=False).mean()

        return signals

    def signal_generation(df):

        signals=macd.macd(df)
        signals['positions']=0
        signals['positions'][ma1:]=np.where(signals['ma1'][ma1:]>=signals['ma2'][ma1:],1,0)
        signals['signals']=signals['positions'].diff()
        signals['oscillator']=signals['ma1']-signals['ma2']

        return signals

    def plot(new, ticker):

        fig = plt.figure(figsize=(20,10))

        ax1 = plt.subplot2grid((2,2), (0,0), colspan=2)
        new['close'].plot(label=ticker)
        plt.plot(new.loc[new['signals']==1].index,new['close'][new['signals']==1],label='LONG',lw=0,marker='^',c='g')
        plt.plot(new.loc[new['signals']==-1].index,new['close'][new['signals']==-1],label='SHORT',lw=0,marker='v',c='r')
        plt.legend(loc='best')
        plt.grid(True)
        plt.title('Positions')

        ax2 = plt.subplot2grid((2,2), (1,0))
        new['oscillator'].plot(kind='bar',color='r')
        plt.legend(loc='best')
        plt.grid(True)
        plt.xticks([])
        plt.xlabel('')
        plt.title('MACD Oscillator')

        ax3 = plt.subplot2grid((2,2), (1,1))
        new['ma1'].plot(label='ma1')
        new['ma2'].plot(label='ma2',linestyle=':')
        plt.legend(loc='best')
        plt.grid(True)

        plt.show()

    def main(df, ticker):

        global ma1
        global ma2

        ma1=40
        ma2=80

        slicer=114670    
        new = df[slicer:].copy()
        new = new.reset_index()

        new = macd.signal_generation(new)
        macd.plot(new, ticker)

## Parabolic SAR

In [84]:
class parabolic_sar:
    def parabolic_sar(new):

        initial_af=0.02
        step_af=0.02
        end_af=0.2


        new['trend']=0
        new['sar']=0.0
        new['real sar']=0.0
        new['ep']=0.0
        new['af']=0.0
        new['trend'][1]=1 if new['close'][1]>new['close'][0] else -1
        new['sar'][1]=new['high'][0] if new['trend'][1]>0 else new['low'][0]
        new.at[1,'real sar']=new['sar'][1]
        new['ep'][1]=new['high'][1] if new['trend'][1]>0 else new['low'][1]
        new['af'][1]=initial_af

        for i in range(2,len(new)):

            temp=new['sar'][i-1]+new['af'][i-1]*(new['ep'][i-1]-new['sar'][i-1])
            if new['trend'][i-1]<0:
                new.at[i,'sar']=max(temp,new['high'][i-1],new['high'][i-2])
                temp=1 if new['sar'][i]<new['high'][i] else new['trend'][i-1]-1
            else:
                new.at[i,'sar']=min(temp,new['low'][i-1],new['low'][i-2])
                temp=-1 if new['sar'][i]>new['low'][i] else new['trend'][i-1]+1
            new.at[i,'trend']=temp


            if new['trend'][i]<0:
                temp=min(new['low'][i],new['ep'][i-1]) if new['trend'][i]!=-1 else new['low'][i]
            else:
                temp=max(new['high'][i],new['ep'][i-1]) if new['trend'][i]!=1 else new['high'][i]
            new.at[i,'ep']=temp


            if np.abs(new['trend'][i])==1:
                temp=new['ep'][i-1]
                new.at[i,'af']=initial_af
            else:
                temp=new['sar'][i]
                if new['ep'][i]==new['ep'][i-1]:
                    new.at[i,'af']=new['af'][i-1]
                else:
                    new.at[i,'af']=min(end_af,new['af'][i-1]+step_af)
            new.at[i,'real sar']=temp

        return new

    def signal_generation(df):

            new = parabolic_sar.parabolic_sar(df)

            new['positions'],new['signals']=0,0
            new['positions']=np.where(new['real sar']<new['close'],1,0)
            new['signals']=new['positions'].diff()

            return new

    def plot(new,ticker):

        fig = plt.figure(figsize=(20,10))

        ax1 = plt.subplot2grid((2,2), (0,0), colspan=2, rowspan=2)
        new['close'].plot(lw=3,label='%s'%ticker)
        new['real sar'].plot(linestyle=':',label='Parabolic SAR',color='k')
        ax1.plot(new.loc[new['signals']==1].index,new['close'][new['signals']==1],marker='^',color='g',label='LONG',lw=0,markersize=10)
        ax1.plot(new.loc[new['signals']==-1].index,new['close'][new['signals']==-1],marker='v',color='r',label='SHORT',lw=0,markersize=10)

        plt.legend(loc='best')
        plt.grid(True)
        plt.title('Parabolic SAR')
        plt.ylabel('price')
        plt.show()

    def main(df, ticker):


        slicer=114670
        new=df[slicer:].copy()
        new = new.reset_index()

        new = parabolic_sar.signal_generation(new)    
        parabolic_sar.plot(new[1:],ticker) 

## Heikin-Ashi

In [87]:
class heikin_ashi:
    def heikin_ashi(df1):

        df1.reset_index(inplace=True)

        df1['HA close']=(df1['open']+df1['close']+df1['high']+df1['low'])/4
        df1['HA open']=float(0)
        df1['HA open'][0]=df1['open'][0]

        for n in range(1,len(df1)):
            df1.at[n,'HA open']=(df1['HA open'][n-1]+df1['HA close'][n-1])/2

        temp=pd.concat([df1['HA open'],df1['HA close'],df1['low'],df1['high']],axis=1)
        df1['HA high']=temp.apply(max,axis=1)
        df1['HA low']=temp.apply(min,axis=1)

        del df1['close']
        del df1['volume']
        return df1

    def signal_generation(df,stls):

        df1 = heikin_ashi.heikin_ashi(df)

        df1['signals']=0
        df1['cumsum']=0

        for n in range(1,len(df1)):

            if (df1['HA open'][n]>df1['HA close'][n] and df1['HA open'][n]==df1['HA high'][n] and
                np.abs(df1['HA open'][n]-df1['HA close'][n])>np.abs(df1['HA open'][n-1]-df1['HA close'][n-1]) and
                df1['HA open'][n-1]>df1['HA close'][n-1]):

                df1.at[n,'signals']=1
                df1['cumsum']=df1['signals'].cumsum()

                if df1['cumsum'][n]>stls:
                    df1.at[n,'signals']=0


            elif (df1['HA open'][n]<df1['HA close'][n] and df1['HA open'][n]==df1['HA low'][n] and 
            df1['HA open'][n-1]<df1['HA close'][n-1]):

                df1.at[n,'signals']=-1
                df1['cumsum']=df1['signals'].cumsum()

                if df1['cumsum'][n]>0:
                    df1.at[n,'signals']=-1*(df1['cumsum'][n-1])

                if df1['cumsum'][n]<0:
                    df1.at[n,'signals']=0

        return df1

    def plot(df1,ticker):

        fig = plt.figure(figsize=(20,10))

        ax1 = plt.subplot2grid((2,1), (0,0))
        mpf.candlestick2_ochl(ax1, df1['HA open'], df1['HA close'], df1['HA high'], df1['HA low'], width=1, colorup='g', colordown='r')
        plt.grid(True)
        plt.xticks([])
        plt.title('Heikin-Ashi')

        ax2 = plt.subplot2grid((2,1), (1,0))
        df1['HA close'].plot(ax=ax2,label=ticker)
        ax2.plot(df1.loc[df1['signals']==1].index,df1['HA close'][df1['signals']==1],marker='^',lw=0,c='g',label='long')
        ax2.plot(df1.loc[df1['signals']<0].index,df1['HA close'][df1['signals']<0],marker='v',lw=0,c='r',label='short')

        plt.grid(True)
        plt.legend(loc='best')
        plt.show()

    def omega(risk_free,degree_of_freedom,maximum,minimum):

        y=integrate.quad(lambda g:1-t.cdf(g,degree_of_freedom),risk_free,maximum)
        x=integrate.quad(lambda g:t.cdf(g,degree_of_freedom),minimum,risk_free)
        z=(y[0])/(x[0])

        return z

    def sortino(risk_free,degree_of_freedom,growth_rate,minimum):

        v=np.sqrt(np.abs(integrate.quad(lambda g:((risk_free-g)**2)*t.pdf(g,degree_of_freedom),risk_free,minimum)))
        s=(growth_rate-risk_free)/v[0]

        return s

    def mdd(series):

        temp=0
        for i in range(1,len(series)):
            if temp>(series[i]/max(series[:i])-1):
                temp=(series[i]/max(series[:i])-1)

        return temp

    def main(df, ticker):

        slicer = 114670
        new=df[slicer:].copy()

        stls = 3
        new = heikin_ashi.signal_generation(new,stls)
        heikin_ashi.plot(new,ticker)